In [18]:
import requests
import os
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import time
import random

user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

def get_data(url,proxy):
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    zhihu_api = f'{url}'
    while True:
        html = requests.get(zhihu_api, headers=headers, proxies = proxy)
        data = html.json()
        if 'error' in data:
            f.write('Retry: ' + str(r) + 'progress:' +str(len(follower_url_token)) +'  '+proxy['http']+ '\n')
            proxy = new_prox()
            r += 1
            continue
        else:
            return data,proxy
    
class follower:
    def __init__(self, locations, educations, business,gender):
        self.locations = locations
        self.educations = educations
        self.business = business
        self.gender = gender
        
    @classmethod
    def from_dic(cls,dic):
        locations, educations, business,gender = dic['locations'], dic['educations'], dic['business'], dic['gender']
        return cls(locations, educations, business, gender)
    
    
# create database
# connection function
def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

def get_tablesql(columns,data):
    comd = ''''''
    for key in columns:
        if data[key] == int or  data[key] == float:
            comd = comd + str(key) + ' NUMBER,'
        else:
            comd = comd + str(key) + ' TEXT,'
    comd = comd[:len(comd)-1]
    return comd

def determine_data_type_of_list(values):
    """
    Function takes a list of strings and determines their data type. 

    """
    ty = []
    if len(values) == 0:
        return 0
    for i in values:
        try:
            x = float(i)
            if x - int(x) == 0:
                ty.append("int")
            else:
                ty.append("float")
        except:
            return type(i)
    if "float" not in ty:
        return int
    else:
        return float

def determine_data_type_of_info_fields(data):
    result = {}
    for key in data:
        lis = data[key]
        result[key] = determine_data_type_of_list(lis)
    return result
    
def create_pdtable(data,table_name):
    dic = {}
    for item in table_name:
        dic[item] = data[item]
    table = pd.DataFrame(dic)
    return table
    
def createTable_insertValue(db_file,columns,pd_table,value_table):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    
    comd = get_tablesql(columns,determine_data_type_of_info_fields(pd_table))
    
    # Create table
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
        %s);'''%(comd))
    
    # insert value
    num_value = ''
    for i in range(len(value_table.keys())):
        num_value += '?,'
    num_value = num_value[:len(num_value)-1]
    c.executemany(f'''insert into users values({num_value});''',(value_table.values))
    
    conn.commit()
    c.close()
    conn.close()

## 代理

In [ ]:
import requests
import base64


def base_code(username, password):
    str = '%s:%s' % (username, password)
    encodestr = base64.b64encode(str.encode('utf-8'))
    return '%s' % encodestr.decode() 

url = "http://myip.ipip.net/"

ip_port = '60.185.76.87:894' # 从api中提取出来的代理IP:PORT
username = 'q'
password = 'XXXXXX'

basic_pwd = base_code(username, password)

headers = {
    'Proxy-Authorization': 'Basic %s' % (base_code(username, password))
}

proxy = {
    'http' : 'socks5://{}'.format(ip_port),
    'https' : 'socks5://{}'.format(ip_port)
}

r = requests.get(url,proxies=proxy, headers=headers)
print(r.text)

In [65]:
daili = 'http://api.wandoudl.com/api/ip?app_key=167c97f43f2ce17e6388159665b15724&pack=0&num=1&xy=1&type=2&lb=\r\n&mr=1&'
html = requests.get(daili, headers=headers)
daili = html.json()

In [66]:
daili

{'code': 200,
 'msg': 'ok',
 'data': [{'ip': '125.87.106.79',
   'port': 5412,
   'expire_time': '2020-05-27 13:13:13',
   'city': '重庆',
   'isp': '电信'}]}

In [25]:
user_agent = random.choice(user_agent_list)
headers = {'User-Agent': user_agent}

In [6]:
def new_prox():
    daili = 'http://api.wandoudl.com/api/ip?app_key=167c97f43f2ce17e6388159665b15724&pack=0&num=1&xy=1&type=2&lb=\r\n&mr=1&'
    html = requests.get(daili, headers=headers)
    daili = html.json()
    porx = daili['data'][0]['ip'] + ':' + str(daili['data'][0]['port'])
    return {'http': 'http://'+porx}

In [7]:
new_prox()

{'http': 'http://61.138.222.177:5412'}

In [22]:
proxy

{'http': 'http://125.123.67.209:766'}

In [96]:
import time
import random
# time.sleep(n)#n是秒，sleep for n seconds

time_interval = random.randint(1,30)# random.uniform(1,5)
time.sleep(time_interval)#时间也可以设置成随机停止，如例子所示为在1到5秒之间随机停止

In [23]:
proxy = {'http': 'http://125.123.67.209:766'}

In [26]:
zhihu_api = 'https://www.zhihu.com/api/v4/members/teng-xun-70?include=follower_count,voteup_count,favorited_count,thanked_count,articles_count'
url = 'http://icanhazip.com/'
html = requests.get(zhihu_api, headers=headers, proxies = proxy)
data = html.text
data

'{"error":{"need_login":true,"redirect":"https:\\/\\/www.zhihu.com\\/account\\/unhuman?type=unhuman&message=%E7%B3%BB%E7%BB%9F%E7%9B%91%E6%B5%8B%E5%88%B0%E6%82%A8%E7%9A%84%E7%BD%91%E7%BB%9C%E7%8E%AF%E5%A2%83%E5%AD%98%E5%9C%A8%E5%BC%82%E5%B8%B8%EF%BC%8C%E4%B8%BA%E4%BF%9D%E8%AF%81%E6%82%A8%E7%9A%84%E6%AD%A3%E5%B8%B8%E8%AE%BF%E9%97%AE%EF%BC%8C%E8%AF%B7%E8%BE%93%E5%85%A5%E9%AA%8C%E8%AF%81%E7%A0%81%E8%BF%9B%E8%A1%8C%E9%AA%8C%E8%AF%81%E3%80%82&need_login=true","code":40352,"message":"系统监测到您的网络环境存在异常，为保证您的正常访问，请输入验证码进行验证。"}}\n'

In [ ]:
def get_random_proxy():
    """
    get random proxy from proxypool
    :return: proxy
    """
    proxy = requests.get(proxypool_url).text.strip()
    proxies = {'http': 'http://' + proxy}
    return proxies

proxypool_url = 'http://localhost:5555/random'
proxy = get_random_proxy()

## 创建数据库

In [ ]:
# Creating or initialize the zhihu.db database
db_file = 'zhihu.db'
conn = create_connection(db_file, delete_db=True)
conn.close()

## 用户资料

In [ ]:
# enter names of company that you want to search
user_id = ['teng-xun-70','bai-du-81-39','a-li-ba-ba-23-79','jing-dong-46-34','da-zhong-dian-ping-83']

user_data = {'id': [], 'url_token': [], 'name': [], 'use_default_avatar': [], 'avatar_url': [], 'avatar_url_template': [], 'is_org': [], 'type': [], 'url': [], 'user_type': [], 'headline': [], 'gender': [], 'is_advertiser': [], 'vip_info': [], 'follower_count': [], 'articles_count': [], 'favorited_count': [], 'voteup_count': [], 'thanked_count': [], 'is_realname': [], 'self_recommend': []}

for i in user_id:
    url = f'https://www.zhihu.com/api/v4/members/{i}?include=follower_count,voteup_count,favorited_count,thanked_count,articles_count'
    data,proxy = get_data(url,proxy)
    for key in data:
        user_data[key].append(data[key])

# select data that we need
col = ['name','url_token','url','follower_count','voteup_count','favorited_count','thanked_count','articles_count']
user_table = create_pdtable(user_data,col) # two par: a dictionary of lists, list of column names that we need

# create table & insert values
createTable_insertValue(db_file,col,user_data,user_table)

In [ ]:
# Load Data
conn = sqlite3.connect(db_file)
c = conn.cursor()

v_1 = pd.DataFrame(c.execute('''select *
                                                        from users;'''),columns = col)

c.close()
conn.close()

In [ ]:
v_1

记录错误


In [19]:
f = open('errors.txt','a')
f2 = open('write.txt','a')
# f.write(str(i) + '\n')

In [17]:
f.close()
f2.close()

## 粉丝资料

In [13]:
def get_followertoken(followers):
    for single_f in followers['data']:
        follower_url_token.append(single_f['url_token'])
        
def get_follower_api(user):
    return user['paging']['next']


In [14]:
follower_data = {'locations': [],'educations': [],'business': [],'gender': []}
follower_url_token = []

In [20]:
url = f'https://www.zhihu.com/api/v4/members/teng-xun-70/followers?limit=10&offset=0'
e = 0
r = 0
s = 0
while int(url.split('=')[-1]) <= 228080:
    try:
        time.sleep(time_interval)
        followers,proxy = get_data(url,proxy) # limit 10 each time
        get_followertoken(followers)
        url = get_follower_api(followers) # get the next api
        if len(follower_url_token)%1000 == 0:
            print(len(follower_url_token))
        f2.write('Success'+str(s) + '\n')
        s += 10
    except:
        if e%1000 == 0 or len(follower_url_token) > 38080:
            print('Error: ',e, 'progress:',len(follower_url_token))
        f.write('Error: ' + str(e) + 'progress:' +str(len(follower_url_token))+ '  '+ proxy['http'] + '\n')
        proxy = new_prox()
        e+=1
        continue

Error:  0 progress: 0


KeyboardInterrupt: 

In [45]:
x

19

In [21]:
len(follower_url_token)

0

In [133]:
# save token
tokens = pd.DataFrame({'token':follower_url_token})
tokens.to_csv('follower_token.csv',index = 'False')

In [ ]:
# get follower data
for token in follower_url_token:
    follower_url = f'https://www.zhihu.com/api/v4/members/{token}?include=locations,educations,business'
    followers,proxy = get_data(url) # limit 10 each time
    followers = follower.from_dic(followers)
    follower_data['locations'].append(followers.locations)
    follower_data['educations'].append(followers.educations)
    follower_data['business'].append(followers.business)
    follower_data['gender'].append(followers.gender)

In [ ]:
len(follower_data['locations'])

In [ ]:
# save follower data
foll = pd.DataFrame(follower_data)
foll.to_csv('follower_token.csv',index = False)

In [ ]:
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    
    comd = get_tablesql(columns,determine_data_type_of_info_fields(pd_table))
    
    # Create table
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
        %s);'''%(comd))
    
    # insert value
    num_value = ''
    for i in range(len(value_table.keys())):
        num_value += '?,'
    num_value = num_value[:len(num_value)-1]
    c.executemany(f'''insert into users values({num_value});''',(value_table.values))
    
    conn.commit()
    c.close()
    conn.close()